# Odległość edycyjna (Levenshtein distance)

In [13]:
import numpy as np
from enum import Enum, auto

In [14]:
class Operations(Enum):
    SWITCH = auto()
    DELETE = auto()
    INSERT = auto()

In [15]:
class LevenshteinDistance:
    def __init__(self, from_word, to_word):
        self.from_word  = from_word
        self.to_word    = to_word
        self.distances  = None
        self.parents    = None
        self.reset_distances_and_parents()
        self.calculate_distances_and_parents()
        self.operations = self.create_operations()
        
    def reset_distances_and_parents(self):
        m = len(self.from_word)
        n = len(self.to_word)
        self.distances = [[0 for _ in range(n+1)] for _ in range(m+1)]
        self.parents   = [[0 for _ in range(n+1)] for _ in range(m+1)]
        for i in range(1, m+1):
            self.distances[i][0] = i
            self.parents[i][0] = (i-1, 0)
        for i in range(1, n+1):
            self.distances[0][i] = i
            self.parents[0][i] = (0, i-1)
    
    def calculate_distances_and_parents(self):
        m = len(self.from_word)
        n = len(self.to_word)
        for i in range(1, m+1):
            for j in range(1, n+1):
                cost = 1
                if self.from_word[i-1] == self.to_word[j-1]:
                    cost = 0

                self.distances[i][j] = min(self.distances[i-1][j]   + 1, 
                                           self.distances[i][j-1] + 1,
                                           self.distances[i-1][j-1] + cost)
                if self.distances[i][j] == self.distances[i-1][j] + 1:
                    self.parents[i][j] = (i-1, j)
                elif self.distances[i][j] == self.distances[i][j-1] + 1:
                    self.parents[i][j] = (i, j-1)
                else:
                    self.parents[i][j] = (i-1, j-1)
    
    def create_operations(self):
        m = len(self.from_word)
        n = len(self.to_word)
        operations = []

        while (m, n) != (0, 0):
            pm, pn = self.parents[m][n]
            d = self.distances[pm][pn]

            if d != self.distances[m][n]:
                if (m-1, n-1) == (pm, pn):
                    operations.append( (Operations.SWITCH, m-1, n-1) )
                elif (m-1, n) == (pm, pn):
                    operations.append( (Operations.DELETE, m-1) )
                else:
                    operations.append( (Operations.INSERT, m, n-1) )
            m, n = pm, pn

        operations.reverse()
        return operations
    
    def visualize(self):
        print(self.from_word, "->", self.to_word)
        idx_diff = 0
        word = self.from_word
        print(word)
        
        for operation in self.operations:
            idx1 = operation[1] + idx_diff
            if operation[0] == Operations.DELETE:
                print(f"DELETE {word[idx1]} FROM INDEX {idx1}")
                print(f"{word[:idx1]}**{word[idx1 + 1:]}")
                word = word[:idx1] + word[idx1 + 1:]
                idx_diff -= 1
            elif operation[0] == Operations.INSERT:
                idx2 = operation[2]
                print(f"INSERT {self.to_word[idx2]} TO INDEX {idx1}")
                print(f"{word[:idx1]}*{self.to_word[idx2]}*{word[idx1:]}")
                word = word[:idx1] + self.to_word[idx2] + word[idx1:]
                idx_diff += 1
            else:
                idx2 = operation[2]
                print(f"SWITCH {word[idx1]} TO {self.to_word[idx2]}")
                print(f"{word[:idx1]}*{self.to_word[idx2]}*{word[idx1 + 1:]}")
                word = word[:idx1] + self.to_word[idx2] + word[idx1 + 1:]
        print(word)
        print("ODLEGŁOŚĆ EDYCYJNA: ", len(self.operations))

### 1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

### 2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym.

### 3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:

* los - kloc

In [16]:
LD1 = LevenshteinDistance("los", "kloc")

In [17]:
LD1.visualize()

los -> kloc
los
INSERT k TO INDEX 0
*k*los
SWITCH s TO c
klo*c*
kloc
ODLEGŁOŚĆ EDYCYJNA:  2


***

* Łódź - Lodz

In [18]:
LD2 = LevenshteinDistance("Łódź", "Lodz")

In [19]:
LD2.visualize()

Łódź -> Lodz
Łódź
SWITCH Ł TO L
*L*ódź
SWITCH ó TO o
L*o*dź
SWITCH ź TO z
Lod*z*
Lodz
ODLEGŁOŚĆ EDYCYJNA:  3


***

* kwintesencja - quintessence

In [20]:
LD3 = LevenshteinDistance("kwintesencja", "quintessence")

In [21]:
LD3.visualize()

kwintesencja -> quintessence
kwintesencja
SWITCH k TO q
*q*wintesencja
SWITCH w TO u
q*u*intesencja
INSERT s TO INDEX 7
quintes*s*encja
SWITCH j TO e
quintessenc*e*a
DELETE a FROM INDEX 12
quintessence**
quintessence
ODLEGŁOŚĆ EDYCYJNA:  5


***

* ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

In [22]:
LD4 = LevenshteinDistance("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

In [23]:
LD4.visualize()

ATGAATCTTACCGCCTCG -> ATGAGGCTCTGGCCCCTG
ATGAATCTTACCGCCTCG
SWITCH A TO G
ATGA*G*TCTTACCGCCTCG
SWITCH T TO G
ATGAG*G*CTTACCGCCTCG
INSERT C TO INDEX 8
ATGAGGCT*C*TACCGCCTCG
SWITCH A TO G
ATGAGGCTCT*G*CCGCCTCG
INSERT G TO INDEX 11
ATGAGGCTCTG*G*CCGCCTCG
DELETE G FROM INDEX 14
ATGAGGCTCTGGCC**CCTCG
DELETE C FROM INDEX 17
ATGAGGCTCTGGCCCCT**G
ATGAGGCTCTGGCCCCTG
ODLEGŁOŚĆ EDYCYJNA:  7


***

### 4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

In [24]:
class LCS:
    def __init__(self, sequence1, sequence2):
        self.sequence1 = sequence1
        self.sequence2 = sequence2
        
    def calculate_LCS(self, get_array=False):
        m = len(self.sequence1)
        n = len(self.sequence2)
        LCS_array = np.zeros((m+1, n+1))
        for i in range(1, m+1):
            for j in range(1, n+1):
                if self.sequence1[i-1] == self.sequence2[j-1]:
                    LCS_array[i, j] = LCS_array[i-1, j-1] + 1
                else:
                    LCS_array[i, j] = max(LCS_array[i, j-1], LCS_array[i-1, j])
        if get_array:
            return LCS_array[m, n], LCS_array
        
        return LCS_array[m, n]

### 5. Korzystając z gotowego tokenizera (np spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.

In [57]:
import spacy
nlp = spacy.load("pl_core_news_sm")

In [58]:
f = open("romeo-i-julia-700.txt", "r", encoding='utf-8')
file = nlp(f.read())
f.close()
tokens = [token.text for token in file]

### 6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [59]:
num_of_tokens_to_delete = int(len(tokens) * 3/100)

In [119]:
version1 = tokens.copy()
version2 = tokens.copy()
for i in range(num_of_tokens_to_delete):
    idx1 = np.random.randint(len(version1))
    while version1[idx1][0] == "\n":
        idx1 = np.random.randint(len(version1))
    version1.pop(idx1)
    
    idx2 = np.random.randint(len(version2))
    while version2[idx2][0] == "\n":
        idx2 = np.random.randint(len(version2))
    version2.pop(idx2)

In [120]:
file_version1 = open("version1.txt", "w", encoding='utf-8')
for i in version1:
    file_version1.write(i)
file_version1.close()

file_version2 = open("version2.txt", "w", encoding='utf-8')
for i in version2:
    file_version2.write(i)
file_version2.close()

### 7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów. Traktujemy teraz token (wyraz) jako podstawową, niepodzielną jednostkę ciągu.

In [121]:
lcs = LCS(version1, version2)

In [122]:
lcs.calculate_LCS()

2542.0

### 8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

In [123]:
def get_lines_from_file(file_name):
    file = open(file_name, "r", encoding='utf-8')
    lines = file.readlines()
    file.close()
    return lines

In [124]:
def diff(file_name_1, file_name_2):
    lines_1 = get_lines_from_file(file_name_1)
    lines_2 = get_lines_from_file(file_name_2)
    result, matrix = LCS(lines_1, lines_2).calculate_LCS(get_array=True)

    m = len(matrix)
    n = len(matrix[0])
    start = (m-1, n-1)
    indexes_1 = [True for i in range(m-1)]
    indexes_2 = [True for i in range(n-1)]

    while start[0] != 0 and start[1] != 0:
        if lines_1[start[0] - 1] == lines_2[start[1] - 1]:
            indexes_1[start[0] - 1] = False
            indexes_2[start[1] - 1] = False
            start = (start[0] - 1, start[1] - 1)
        elif matrix[start[0] - 1, start[1]] == matrix[start]:
            start = (start[0] - 1, start[1])
        else:
            start = (start[0], start[1] - 1)
            
    file_1_diffs = []
    file_2_diffs = []

    for i in range(len(indexes_1)):
        if indexes_1[i]:
            file_1_diffs.append((lines_1[i], i))

    for i in range(len(indexes_2)):
        if indexes_2[i]:
            file_2_diffs.append((lines_2[i], i))
            
    for x, y in file_1_diffs:
        print(f"plik 1, numer linii {y}: {x}")
    for x, y in file_2_diffs:
        print(f"plik 2, numer linii {y}: {x}")   

### 9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [125]:
diff("version1.txt", "version2.txt")

plik 1, numer linii 5: ISBN978-83-2882903-9

plik 1, numer linii 12:  *MONTEKI,KAPULET—dwóchdomównieprzyjaznychsobie

plik 1, numer linii 18:  *LAURENTY—ojciecfranciszkanin

plik 1, numer linii 19:  *JAN—bratzzgromadzenia

plik 1, numer linii 20:  *BALTAZAR—służącyRomea

plik 1, numer linii 21:  *SAMSON,—słudzyKapuleta

plik 1, numer linii 23:  *

plik 1, numer linii 29:  *PANIKAPULET—małżonkaKapuleta

plik 1, numer linii 31:  *MARTA—mamkaJulii

plik 1, numer linii 32:  *Obywateleweroneńscy,różneosobypłciobojej,liczącydoprzyjaciółdomów,maski,strażwojskowaiinneosoby.

plik 1, numer linii 37: Rzeczodbywasięprzezwiększączęśćsztukiw,przezczęśćpiątegoaktuwMantui.

plik 1, numer linii 45: Dwarody,zacnejednakosławne—

plik 1, numer linii 46: ,gdziesięrzecztarozgrywa,wWeronie,

plik 1, numer linii 50: Złontychdwuwrogówwzięłobowiemżycie,

plik 1, numer linii 55: Tejichmiłościprzebiegzbytbolesny

plik 1, numer linii 61: Jestwnimcozłego,myusuniembłędy

plik 1, numer linii 72: Placpubliczny.Wchodz